# 02_preprocessing.ipynb – Datenbereinigung und Preprocessing

**Ziel:** Die Amazon Reviews Daten werden bereinigt, normalisiert und als preprocessed Datensätze gespeichert.

---


## 1. Setup und Laden der Rohdaten
Wir laden die Rohdaten aus dem Ordner `data/raw`.


In [4]:
import pandas as pd
import json
import re
from pathlib import Path

RAW_DIR = Path('../data/raw')
PRE_DIR = Path('../data/preprocessed')
PRE_DIR.mkdir(parents=True, exist_ok=True)

CATEGORIES = ['Automotive', 'Video_Games', 'Books']
dfs = {}
N = 1_000_000            # Zeilenlimit pro Kategorie

for cat in CATEGORIES:
    data = []
    with open(RAW_DIR / f'{cat}.jsonl', 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= N:          # nach N Zeilen abbrechen
                break
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    dfs[cat] = df
    print(f'{cat}: {df.shape[0]} Einträge geladen.')


Automotive: 1000000 Einträge geladen.
Video_Games: 1000000 Einträge geladen.
Books: 1000000 Einträge geladen.


## 2. Bereinigung: Leere Texte, ungültige Ratings, Duplikate entfernen
Wir filtern alle Rezensionen mit leerem Text, Rating außerhalb 1–5 und entfernen Duplikate.


In [5]:
def clean_df(df):
    # Entferne Ratings außerhalb 1–5
    df = df[df['rating'].isin([1, 2, 3, 4, 5])]
    # Entferne leere Texte
    df = df[df['text'].astype(str).str.strip() != '']
    # Entferne Duplikate
    if set(['user_id', 'asin', 'text']).issubset(df.columns):
        df = df.drop_duplicates(subset=['user_id', 'asin', 'text'])
    return df.reset_index(drop=True)

for cat in CATEGORIES:
    before = dfs[cat].shape[0]
    dfs[cat] = clean_df(dfs[cat])
    after = dfs[cat].shape[0]
    print(f'{cat}: {before} → {after} nach Bereinigung.')


Automotive: 1000000 → 996426 nach Bereinigung.
Video_Games: 1000000 → 995507 nach Bereinigung.
Books: 1000000 → 997498 nach Bereinigung.


## 3. Textnormalisierung: Kleinbuchstaben, Sonderzeichen entfernen
Wir wandeln alle Texte in Kleinbuchstaben um und entfernen Sonderzeichen.


In [6]:
def normalize_text(text):
    text = str(text).lower()
    text = re.sub(r'<.*?>', ' ', text)      # HTML-Tags
    text = re.sub(r'[^\w\s]', ' ', text)    # Sonderzeichen
    text = re.sub(r'\d+', ' ', text)        # Zahlen
    text = re.sub(r'\s+', ' ', text).strip()
    return text

for cat in CATEGORIES:
    dfs[cat]['text'] = dfs[cat]['text'].apply(normalize_text)
    dfs[cat]['title'] = dfs[cat]['title'].apply(normalize_text)
    print(f'{cat}: Texte normalisiert.')


Automotive: Texte normalisiert.
Video_Games: Texte normalisiert.
Books: Texte normalisiert.


## 4. Speichern der bereinigten Datensätze
Wir speichern die bereinigten Daten als .jsonl-Dateien im Ordner `data/preprocessed`.



In [8]:
for cat, df in dfs.items():
    out_path = PRE_DIR / f'{cat}_preprocessed.jsonl'
    df.to_json(out_path, orient='records', lines=True, force_ascii=False)
    print(f'{cat}: gespeichert unter {out_path}')


Automotive: gespeichert unter ..\data\preprocessed\Automotive_preprocessed.jsonl
Video_Games: gespeichert unter ..\data\preprocessed\Video_Games_preprocessed.jsonl
Books: gespeichert unter ..\data\preprocessed\Books_preprocessed.jsonl


---
**Hinweis:** Weitere Schritte wie Lemmatisierung, Stopword-Entfernung oder Tokenisierung können später ergänzt werden, z. B. mit spaCy.
